In [ ]:
In This notebook, you will find the code to scrape the data of the website transfert market.

# Modules

In [27]:
import urllib
import bs4
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import urllib
import bs4
from bs4 import BeautifulSoup
from urllib import request
import pandas as pd

In [29]:
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("enable-automation")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument("--disable-gpu")

In [30]:
driver = webdriver.Chrome('C:/Users/Suvin/Downloads/chromedriver_win32/chromedriver.exe', options = options)

# TRANFERTMARKT

In [31]:
transfert_markt = "https://www.transfermarkt.fr/wettbewerbe/europa"

In [32]:
driver.get(transfert_markt)
html = driver.page_source
print(driver.title)

Championnats et coupes européens | Transfermarkt


## 1) Scrape URL of 5 big league

In [33]:
nom_championnat =driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr/td/table/tbody/tr/td[2]//a')
url_championnat =driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr/td/table/tbody/tr/td[2]//a')
nombre_club = driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr/td[3]')

leagues =[]

for i in range(0,5):
    temporary={'nom_championnat':nom_championnat[i].get_attribute('title'),
                           'url_championnat':url_championnat[i].get_attribute('href'),
                           'nombre_club':int(nombre_club[i].get_attribute('innerText'))     }
    leagues.append(temporary)

#driver.quit()
df_leagues = pd.DataFrame(leagues)
df_leagues.head()

,nom_championnat,url_championnat,nombre_club
0,Premier League,https://www.transfermarkt.fr/premier-league/st...,20
1,LaLiga,https://www.transfermarkt.fr/laliga/startseite...,20
2,Serie A,https://www.transfermarkt.fr/serie-a/startseit...,20
3,Bundesliga,https://www.transfermarkt.fr/bundesliga/starts...,18
4,Ligue 1,https://www.transfermarkt.fr/ligue-1/startseit...,20


## 2) Scrape URL of every club of every league

In [40]:
clubs =[]
for championnat in range(0,5):
        league = df_leagues['url_championnat'][championnat]
        driver.get(league)
        nom_championnat = df_leagues['nom_championnat'][championnat]
        
        nom_club =driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr/td[3]/a[1]')
        url_club =driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr/td[2]/a[1]')
        nombre_joueur = driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr/td[3]/a[1]')
        
        
        stop_range=df_leagues['nombre_club'][championnat]
        
        for i in range(0,stop_range):
            temporary={'nom_championnat':nom_championnat,
                'nom_club':nom_club[i].get_attribute('title'),
                           'url_club':url_club[i].get_attribute('href'),
                           'nombre_joueur':int(nombre_joueur[i].get_attribute('innerText'))     }
            clubs.append(temporary)
            
df_clubs = pd.DataFrame(clubs)
df_clubs.head(100)

,nom_championnat,nom_club,url_club,nombre_joueur
0,Premier League,Manchester City,https://www.transfermarkt.fr/manchester-city/s...,24
1,Premier League,FC Liverpool,https://www.transfermarkt.fr/fc-liverpool/star...,28
2,Premier League,Chelsea FC,https://www.transfermarkt.fr/fc-chelsea/starts...,25
3,Premier League,FC Arsenal,https://www.transfermarkt.fr/fc-arsenal/starts...,23
4,Premier League,Manchester United,https://www.transfermarkt.fr/manchester-united...,28
...,...,...,...,...
93,Ligue 1,Stade Brestois 29,https://www.transfermarkt.fr/stade-brest-29/st...,24
94,Ligue 1,Toulouse FC,https://www.transfermarkt.fr/fc-toulouse/start...,26
95,Ligue 1,Clermont Foot 63,https://www.transfermarkt.fr/clermont-foot-63/...,26
96,Ligue 1,AJ Auxerre,https://www.transfermarkt.fr/aj-auxerre/starts...,31


## 3) Scrape url of every player of every clubs

In [42]:
joueurs =[]

for club in range(0,len(df_clubs)):
    equipe = df_clubs['url_club'][club]
    driver.get(equipe)
    nom_championnat = df_clubs['nom_championnat'][club]
    nom_club = df_clubs['nom_club'][club]
    
    nom_joueur =driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr/td[2]/table/tbody/tr/td[2]/div[1]/span//a')
    url_joueur =driver.find_elements_by_xpath('//*[@id="yw1"]/table/tbody/tr/td[2]/table/tbody/tr/td[2]/div[1]/span//a')
    
    stop_range=df_clubs['nombre_joueur'][club]

    for i in range(0,stop_range):
            temporary={'nom_championnat':nom_championnat,
                       'nom_club':nom_club,
                'nom_joueur':nom_joueur[i].get_attribute('title'),
                           'url_joueur':url_joueur[i].get_attribute('href')    }
            joueurs.append(temporary)
df_joueurs = pd.DataFrame(joueurs)

## 4) OPTIONNAL : Save Results 

In [45]:
df_joueurs.to_csv('C:/Users/Suvin/Desktop/Web Scrapping/url_joueurs.csv',index=False)

In [115]:
df = pd.read_csv("C:/Users/Suvin/Desktop/Web Scrapping/url_joueurs.csv",sep=',')

In [116]:
df.head(10)

,nom_championnat,nom_club,nom_joueur,url_joueur
0,Premier League,Manchester City,Ederson,https://www.transfermarkt.fr/ederson/profil/sp...
1,Premier League,Manchester City,Stefan Ortega,https://www.transfermarkt.fr/stefan-ortega/pro...
2,Premier League,Manchester City,Scott Carson,https://www.transfermarkt.fr/scott-carson/prof...
3,Premier League,Manchester City,Rúben Dias,https://www.transfermarkt.fr/ruben-dias/profil...
4,Premier League,Manchester City,Aymeric Laporte,https://www.transfermarkt.fr/aymeric-laporte/p...
5,Premier League,Manchester City,John Stones,https://www.transfermarkt.fr/john-stones/profi...
6,Premier League,Manchester City,Nathan Aké,https://www.transfermarkt.fr/nathan-ake/profil...
7,Premier League,Manchester City,Manuel Akanji,https://www.transfermarkt.fr/manuel-akanji/pro...
8,Premier League,Manchester City,Sergio Gómez,https://www.transfermarkt.fr/sergio-gomez/prof...
9,Premier League,Manchester City,Benjamin Mendy,https://www.transfermarkt.fr/benjamin-mendy/pr...


## 5) Scrapping of needed data for every players

In [ ]:
age = //*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[8]
taille = //*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[10]
nationalité = //*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[12]/img
poste = //*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[14]
pied = //*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[16]
equipe_actuel_depuis = //*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[22]
fin_contrat=//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[24]
valeur_marchande=//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[1]/div[2]

In [167]:
data =[]
for player in range(0,len(df)):
    joueur = df['url_joueur'][player]
    driver.get(joueur)
    
    nom_championnat = df['nom_championnat'][player]
    nom_club = df['nom_club'][player]
    nom_joueur = df['nom_joueur'][player]
    try:
        time.sleep(1)
        debut =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[1]')
        debut2 =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[3]')
    
        if (debut.get_attribute('innerText') !="Date de naissance:" and debut2.get_attribute('innerText') !="Date de naissance:" ):        
            age =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[10]')
            taille =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[12]')
            nationalite =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[14]/img[1]')
            poste =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[16]')
            pied =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[18]')
            debut_equipe_actuel =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[24]')
            fin_contrat =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[26]')
            valeur_marchande =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[1]/div[2]')
    
            temporary={'nom_championnat':nom_championnat,
                       'nom_club':nom_club,
                'nom_joueur':nom_joueur,
                           'age':age.get_attribute('innerText'),
                 'taille':taille.get_attribute('innerText'),
                 'nationalite':nationalite.get_attribute('title'),
                 'poste':poste.get_attribute('innerText'),
                 'debut_equipe_actuel':debut_equipe_actuel.get_attribute('innerText'),
                 'fin_contrat':fin_contrat.get_attribute('innerText'),
               'valeur_marchande':valeur_marchande.get_attribute('innerText')               
               }
            data.append(temporary)
        
        
        elif (debut.get_attribute('innerText') !="Date de naissance:"):        
            age =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[8]')
            taille =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[10]')
            nationalite =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[12]/img[1]')
            poste =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[14]')
            pied =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[16]')
            debut_equipe_actuel =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[22]')
            fin_contrat =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[24]')
            valeur_marchande =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[1]/div[2]')
    
            temporary={'nom_championnat':nom_championnat,
                       'nom_club':nom_club,
                'nom_joueur':nom_joueur,
                           'age':age.get_attribute('innerText'),
                 'taille':taille.get_attribute('innerText'),
                 'nationalite':nationalite.get_attribute('title'),
                 'poste':poste.get_attribute('innerText'),
                 'debut_equipe_actuel':debut_equipe_actuel.get_attribute('innerText'),
                 'fin_contrat':fin_contrat.get_attribute('innerText'),
               'valeur_marchande':valeur_marchande.get_attribute('innerText')               
               }
            data.append(temporary)   
        
        elif (debut.get_attribute('innerText') =="Date de naissance:" ):
            age =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[6]')
            taille =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[8]')
            nationalite =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[10]/img[1]')
            poste =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[12]')
            pied =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[14]')
            debut_equipe_actuel =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[20]')
            fin_contrat =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[2]/div/span[22]')
            valeur_marchande =driver.find_element_by_xpath('//*[@id="main"]/main/div[3]/div[1]/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[1]/div[2]')
    
            temporary={'nom_championnat':nom_championnat,
                       'nom_club':nom_club,
                'nom_joueur':nom_joueur,
                           'age':age.get_attribute('innerText'),
                 'taille':taille.get_attribute('innerText'),
                 'nationalite':nationalite.get_attribute('title'),
                 'poste':poste.get_attribute('innerText'),
                 'debut_equipe_actuel':debut_equipe_actuel.get_attribute('innerText'),
                 'fin_contrat':fin_contrat.get_attribute('innerText'),
               'valeur_marchande':valeur_marchande.get_attribute('innerText')               
               }
            data.append(temporary)
    except Exception:
        pass
dataframe =  pd.DataFrame(data)

In [169]:
dataframe.tail(100)

,nom_championnat,nom_club,nom_joueur,age,taille,nationalite,poste,debut_equipe_actuel,fin_contrat,valeur_marchande
2446,Ligue 1,Stade Brestois 29,Franck Honorat,26,"1,80 m",France,Attaquant - Ailier droit,1 juil. 2020,30 juin 2025,"9,00 mio. €"
2447,Ligue 1,Stade Brestois 29,Romain Del Castillo,26,"1,72 m",France,Attaquant - Ailier droit,31 août 2021,30 juin 2024,"3,00 mio. €"
2448,Ligue 1,Stade Brestois 29,Karamoko Dembélé,19,"1,67 m",Angleterre,Attaquant - Ailier droit,5 juil. 2022,30 juin 2026,500 K €
2449,Ligue 1,Stade Brestois 29,Steve Mounié,28,"1,90 m",Bénin,Attaquant - Avant-centre,9 sept. 2020,30 juin 2024,"6,00 mio. €"
2450,Ligue 1,Stade Brestois 29,Jérémy Le Douaron,24,"1,89 m",France,Attaquant - Avant-centre,1 juil. 2020,30 juin 2026,"3,00 mio. €"
...,...,...,...,...,...,...,...,...,...,...
2541,Ligue 1,AC Ajaccio,Riad Nouri,37,"1,78 m",France,Attaquant - Ailier droit,7 sept. 2020,30 juin 2024,250 K €
2542,Ligue 1,AC Ajaccio,Mounaïm El Idrissy,23,"1,81 m",France,Attaquant - Avant-centre,1 juil. 2019,30 juin 2023,"1,80 mio. €"
2543,Ligue 1,AC Ajaccio,Bevic Moussiti-Oko,27,"1,84 m",Congo,Attaquant - Avant-centre,10 juil. 2020,30 juin 2023,700 K €
2544,Ligue 1,AC Ajaccio,Romain Hamouma,35,"1,79 m",France,Attaquant - Avant-centre,1 juil. 2022,30 juin 2023,600 K €


In [170]:
dataframe.to_csv('C:/Users/Suvin/Desktop/Web Scrapping/data_joueurs.csv',index=False)